In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize

In [2]:
# Clase que representa la neurona basada en el modelo BMS
class BMS:
    # Función para generar los trenes de pulso y las tasas de disparo
    def __init__(self, sim_time=1000, gamma=0.5, theta=1):
        self.sim_time = sim_time
        self.gamma = gamma
        self.theta = theta

    def simulate(self, i_ext):
        v = []  # Vector de voltaje de membrana
        spike_train = []  # Tren de pulsos

        v.append(0.0)  # Inicializamos el vector de voltaje con 0

        # Ciclo para obtener el tren de pulsos
        for k in range(1, self.sim_time):
            # Operación para obtener el valor del voltaje
            _v = self.gamma * v[-1] * (1 - (1 if v[-1] >= self.theta else 0)) + i_ext

            # Comprobar si la neurona descanso
            if _v >= self.theta:
                spike_train.append(k)
                _v = self.theta  # Ajustamos el voltaje al valor del umbral

            v.append(_v)  # Agregamos el voltaje

        # Devolvemos el tren de pulsos y la tasa de disparo
        return spike_train, (len(spike_train) / self.sim_time)

In [3]:
#Clase que representa la metaheurística que proporciona el vector de pesos
class BMS_Training(Problem):
    def __init__(self, X, y, sim_time, gamma, theta, n_var, xl, xu):
        super().__init__(n_var=n_var, xl=xl, xu=xu)
        self.X = X
        self.y = y
        self.neuron = BMS(sim_time, gamma, theta)
        self.n_class = len(np.unique(y))

    def _evaluate(self, x, out, *args, **kwargs):
        fit = np.zeros(len(x))
        firing_rates = {i: [] for i in range(self.n_class)}

        for i, w in enumerate(x):
            # Ciclo para obtener las tasas de disparo por cada clase
            for _x, _y in zip(self.X, self.y):
                _iext = (
                    _x @ w
                )  # Generar el estímulo a partir del patron actual y los pesos
                _, _fr = self.neuron.simulate(
                    _iext
                )  # Obtener la cantidad de disparos que hizo el patrón actual
                firing_rates[_y].append(
                    _fr
                )  # Agregar la tasa de disparo del patrón a la clase correspondiente

            _m = np.zeros(self.n_class)
            _sd = np.zeros(self.n_class)

            # Ciclo para obtener la media y la desviacón estándar por cada clase
            for _k in firing_rates.keys():
                _m[_k] = np.mean(firing_rates[_k])
                _sd[_k] = np.std(firing_rates[_k])

            # Ciclo para obtener la distancia entre las tasas de disparo por clase
            sum_dist = 0
            for j in range(len(_m)):
                for k in range(j + 1, len(_m)):
                    # tmp = _m[j] - _m[k]
                    # sum_dist += np.sqrt(np.dot(tmp, tmp))
                    sum_dist += np.linalg.norm(_m[j] - _m[k])

            # Obtener la aptitud de cada clase
            fit[i] = ((1 / sum_dist) if sum_dist > 0 else 1000000) + (np.sum(_sd) if np.sum(_sd) > 0 else 1000000)

        out["F"] = fit


In [4]:
# Clase que representa el clasificador y nos da las predicciones
class BMS_Classifier(BaseEstimator):
    def __init__(self, sim_time=100, gamma=0.5, theta=1):
        self.sim_time = sim_time
        self.gamma = gamma
        self.theta = theta
        self.neuron = BMS(self.sim_time, self.theta, self.gamma)
        self._m = None
        self.n_class = None

    def fit(self, X, y):
        problem = BMS_Training(
            X,
            y,
            self.sim_time,
            self.gamma,
            self.theta,
            n_var=X.shape[1],
            xl=np.repeat(-1, X.shape[1]),
            xu=np.repeat(1, X.shape[1]),
        )

        algorithm = DE(
            sampling=LHS(),
            variant="DE/rand/1/bin",
            pop_size=30,
            CR=0.3,
            dither="vector",
            jitter=False,
        )

        solution = minimize(
            problem=problem,
            algorithm=algorithm,
            termination=("n_gen", 10),
        )

        self.n_class = len(np.unique(y))
        self.w = solution.X

        firing_rates = {i: [] for i in range(len(np.unique(y)))}

        for _x, _y in zip(X, y):
            _iext = (
                _x @ self.w
            )  # Generar el estímulo a partir del patron actual y los pesos
            _, _fr = self.neuron.simulate(
                _iext
            )  # Obtener la cantidad de disparos que hizo el patrón actual
            firing_rates[_y].append(
                _fr
            )  # Agregar la tasa de disparo del patrón a la clase correspondiente
        self._m = np.zeros(len(np.unique(y)))

        for _k in firing_rates.keys():
            self._m[_k] = np.mean(firing_rates[_k])

    def predict(self, X):
        pred = np.zeros(X.shape[0], dtype=int)
        for j, x in enumerate(X):
            _iext = x @ self.w
            _, _fr = self.neuron.simulate(_iext)
            dif = np.zeros(self.n_class)

            for i, m in enumerate(self._m):
                dif[i] = np.abs(_fr - m)

            pred[j] = np.argmin(dif)

        return pred


In [ ]:
import load_datasets as datasets


def main():
    datasets_names = []

    with open("datasets_loading.txt", mode="r") as file:
        for line in file:
            line = line.replace("\n", "")
            datasets_names.append(line)

    file.close()

    # Entrenador Base
    bmsC = BMS_Classifier(gamma=0.5)

    # Clasificador Bagging
    bmsE = BaggingClassifier(base_estimator=bmsC, n_estimators=4, n_jobs=-1)

    # Diccionario para guardar los valores obtenidos por cada dataset
    results = {i: {} for i in range(len(datasets_names))}

    # Ciclo para probar todos los datasets
    for i, a_dataset in enumerate(datasets_names):
        X, y = eval(a_dataset)

        # Separar en conjunto de entrenamiento y de prueba
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, stratify=y)

        # Entrenamiento con un solo clasificador
        bmsC.fit(X_tr, y_tr)
        bmsC_pred = bmsC.predict(X_te)
        acc_classifier = round(accuracy_score(bmsC_pred, y_te), 2)
        results[i] = {"Classifier": acc_classifier}

        # Entrenamiento con varios clasificadores(Bagging Ensemble)
        bmsE.fit(X_tr, y_tr)
        bmsE_pred = bmsE.predict(X_te)
        acc_bagging = round(accuracy_score(bmsE_pred, y_te), 2)
        results[i].update({"Bagging": acc_bagging})

    # Ciclo para imprimir los resultados
    for key, values in results.items():
        print(f"Dataset #{key+1} | Accuracies: {values}")


main()